In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 16:05:23 WARN Utils: Your hostname, farik-ThinkPad-E14-Gen-2 resolves to a loopback address: 127.0.1.1; using 192.168.8.105 instead (on interface wlp0s20f3)
24/03/03 16:05:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 16:05:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.createOrReplaceTempView('green')

In [18]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,


    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")


In [19]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 11:00:00|  41| 513.6600000000003|            46|
|2020-01-30 13:00:00|  75| 1330.859999999999|            76|
|2020-01-27 20:00:00| 244|398.00000000000006|            16|
|2020-01-16 10:00:00|  74|1021.2199999999987|            80|
|2020-01-24 17:00:00| 260| 308.0600000000001|            27|
|2020-01-03 01:00:00|  80|             80.61|             6|
|2020-01-08 14:00:00|  33| 687.7900000000001|            34|
|2020-01-07 19:00:00|  66|324.78000000000003|            19|
|2020-01-14 12:00:00|  21|             90.22|             2|
|2020-01-17 17:00:00|  41| 845.2499999999994|            66|
|2020-01-02 09:00:00|   7|367.50000000000017|            30|
|2020-01-15 20:00:00| 145|             39.41|             2|
|2020-01-06 19:00:00|  92| 293.2000000000001|            18|
|2020-01-24 11:00:00| 12

In [30]:
df_green_revenue \
    .repartition(8) \
    .write.parquet('data/report/revenue/green', mode="overwrite")

24/03/03 16:21:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [21]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [22]:
df_yellow.createOrReplaceTempView('yellow')

In [25]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,


    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")


In [26]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 455.1700000000001|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|106.99999999999999|             6|
|2020-01-01 00:00:00|  13|1214.8000000000002|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.20999999999998|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24|            754.95|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [31]:
df_yellow_revenue \
    .repartition(8) \
    .write.parquet('data/report/revenue/yellow', mode="overwrite")

24/03/03 16:21:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [48]:
df_green_revenue = spark.read.parquet('data/report/revenue/green/')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow/')

In [49]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed("amount", "green_amount") \
    .withColumnRenamed("number_records", "green_number_records")

In [50]:
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed("amount", "yellow_amount") \
    .withColumnRenamed("number_records", "yellow_number_records")

In [51]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=["hour", "zone"], how="outer")

In [41]:
df_join.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   7|769.7299999999997|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  13|             null|                null|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|              7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  32|68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  34|             null|                null|              19.3|                    1|
|2020-01-01 00:00:00|  83|94.09999999999998|                   7|               9.8|                    1|
|2020-01-01 00:00:00|  90|           

In [52]:
df_join.write.parquet("data/report/revenue/total", mode="overwrite")

24/03/03 17:00:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [53]:
df_join = spark.read.parquet('data/report/revenue/total')

In [54]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  24|              87.6|                   3|            754.95|                   45|
|2020-01-01 00:00:00|  36|            295.34|                  11|            109.17|                    3|
|2020-01-01 00:00:00|  59|50.900000000000006|                   3|              null|                 null|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|
|2020-01-01 00:00:00|  65|199.48999999999998|                  10| 409.3500000000001|                   19|
|2020-01-01 00:00:00|  88|              null|                null| 823.8000000000002|                   36|
|2020-01-01 00:00:00|  93|  

In [56]:
df_zones = spark.read.parquet('zones/')

In [61]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [66]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

24/03/03 17:05:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
